Webscraping


In [5]:
from IPython.display import display
import json
import pandas as pd
pd.options.display.max_columns = None # Display all columns of a dataframe
pd.options.display.max_rows = 700
from pprint import pprint
import re
import requests
from bs4 import BeautifulSoup
#test coucouc

In [6]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

chromeOptions = webdriver.ChromeOptions()
#it is way to add options

path = 'C:/Users/User/Documents/Github/Films_DM_2023/chromedriver_win32/' #the path
driver = webdriver.Chrome(executable_path=os.path.join(path, 'chromedriver'))

driver.get("https://www.allocine.fr/")
time.sleep(1) # Let the user actually see something!

C:\Users\User\AppData\Local\Temp\ipykernel_42572\2225407633.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=os.path.join(path, 'chromedriver'))
Unable to obtain driver using Selenium Manager: C:\Users\User\anaconda3\envs\datamanagement\lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe is missing.  Please open an issue on https://github.com/SeleniumHQ/selenium/issues


SeleniumManagerException: Message: C:\Users\User\anaconda3\envs\datamanagement\lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe is missing.  Please open an issue on https://github.com/SeleniumHQ/selenium/issues


In [7]:
#Accepter les cookies pour pouvoir passer à la suite
cookies = driver.find_element(By.CLASS_NAME, "jad_cmp_paywall_button-cookies").click()
time.sleep(2)


NameError: name 'driver' is not defined

In [16]:
#Aller dans films 
films = driver.find_element(By.XPATH, '//*[@title="Cinéma"]').click()
time.sleep(2)

In [17]:
#Aller dans films 
tous_films = driver.find_element(By.XPATH, '//*[@title="Tous les films"]').click()
time.sleep(2)


In [6]:
# Assuming you already have the driver set up
# driver = webdriver.Chrome(executable_path='path/to/chromedriver')

# Corrigé par Adin -> Il faudrait créer une loop qui modifie le numéro ce qui marche
#wait = WebDriverWait(driver, 20)
#action = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[@href="/films/genre-13027/"]')))
#action.click()

In [8]:
#Choisir les genres
#action =driver.find_element(By.XPATH,'//*[@class="filter-entity-select"]/div[1]/select/option[2]').click()

In [37]:
#Avoir feuille par feuille
films =[]
for each_fiche in driver.find_elements(By.CLASS_NAME,'mdl'):
    #titres
    titre=each_fiche.find_element(By.CLASS_NAME, "meta-title-link").text
    #date sortie
    date_sortie= each_fiche.find_element(By.CLASS_NAME, "date").text
    #durée
    duree = each_fiche.find_element(By.XPATH, './div[@class="meta-body-info"]').text
    #duree = each_fiche.find_element(By.CLASS_NAME, "meta-body-info").text
    #duree = each_fiche.find_element(By.CLASS_NAME, "spacer").text
    #genre1
    genre1= each_fiche.find_element(By.CLASS_NAME, "xXx").text #faut réussir à mettre plusieurs genres
    #genre2
    genre2= each_fiche.find_element(By.CLASS_NAME, "xXx").text #faut réussir à mettre plusieurs genres
    #réalisateur
    realisateur= each_fiche.find_element(By.CLASS_NAME, "blue-link").text
    #évaluation presse
    #note_presse= each_fiche.find_element(By.XPATH, "./*[@class='rating-holder-3']")# /div[1]/div/div/span").text
    
    films.append({
                "Titre": titre,
                "date_sortie" : date_sortie,
                "duree": duree,
                "genre1": genre1,
                "genre2": genre2,
                "réalisateur": realisateur,
                #"note presse": note_presse,
            })
#dataframe
films_data = pd.DataFrame(films)
films_data

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"./div[@class="meta-body-info"]"}
  (Session info: chrome=111.0.5563.147)
Stacktrace:
Backtrace:
	(No symbol) [0x00D637D3]
	(No symbol) [0x00CF8B81]
	(No symbol) [0x00BFB36D]
	(No symbol) [0x00C2D382]
	(No symbol) [0x00C2D4BB]
	(No symbol) [0x00C22C41]
	(No symbol) [0x00C4B464]
	(No symbol) [0x00C22B96]
	(No symbol) [0x00C4B774]
	(No symbol) [0x00C61215]
	(No symbol) [0x00C4B216]
	(No symbol) [0x00C20D97]
	(No symbol) [0x00C2253D]
	GetHandleVerifier [0x00FDABF2+2510930]
	GetHandleVerifier [0x01008EC1+2700065]
	GetHandleVerifier [0x0100C86C+2714828]
	GetHandleVerifier [0x00E13480+645344]
	(No symbol) [0x00D00FD2]
	(No symbol) [0x00D06C68]
	(No symbol) [0x00D06D4B]
	(No symbol) [0x00D10D6B]
	BaseThreadInitThunk [0x77277D69+25]
	RtlInitializeExceptionChain [0x7788BB9B+107]
	RtlClearBits [0x7788BB1F+191]
